In [13]:
import numpy as np
import matplotlib.pyplot as  plt
import pandas as pd

In [14]:
df=pd.read_csv('crop_yield_m.csv')
df

,Crop,Crop_Year,Season,State,Area,Production,Annual_Rainfall,Fertilizer,Pesticide,Yield
0,Arecanut,1997,Whole Year,Assam,73814.0,56708,2051.4,7024878.38,22882.34,0.796087
1,Arhar/Tur,1997,Kharif,Assam,6637.0,4685,2051.4,631643.29,2057.47,0.710435
2,Castor seed,1997,Kharif,Assam,796.0,22,2051.4,75755.32,246.76,0.238333
3,Coconut,1997,Whole Year,Assam,19656.0,126905000,2051.4,1870661.52,6093.36,5238.051739
4,Cotton(lint),1997,Kharif,Assam,1739.0,794,2051.4,165500.63,539.09,0.420909
...,...,...,...,...,...,...,...,...,...,...
19684,Small millets,1998,Kharif,Nagaland,4000.0,2000,1498.0,395200.00,1160.00,0.500000
19685,Wheat,1998,Rabi,Nagaland,1000.0,3000,1498.0,98800.00,290.00,3.000000
19686,Maize,1997,Kharif,Jammu and Kashmir,310883.0,440900,1356.2,29586735.11,96373.73,1.285000
19687,Rice,1997,Kharif,Jammu and Kashmir,275746.0,5488,1356.2,26242746.82,85481.26,0.016667


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Crop             19689 non-null  object 
 1   Crop_Year        19689 non-null  int64  
 2   Season           19689 non-null  object 
 3   State            19689 non-null  object 
 4   Area             19689 non-null  float64
 5   Production       19689 non-null  int64  
 6   Annual_Rainfall  19689 non-null  float64
 7   Fertilizer       19689 non-null  float64
 8   Pesticide        19689 non-null  float64
 9   Yield            19689 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 1.5+ MB


In [16]:
X = df.drop(columns=['Crop','Yield','Fertilizer','Pesticide'])
Y = df['Crop'] 
print(X.shape,Y.shape)


(19689, 6) (19689,)


In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,Y_train.shape)

(15751, 6) (15751,)


In [18]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
# Y_train['Crop'] = label_encoder.fit_transform(Y_train['Crop'])
X_train['Season'] = label_encoder.fit_transform(X_train['Season'])
X_train['State'] = label_encoder.fit_transform(X_train['State'])

# Y_test['Crop'] = label_encoder.fit_transform(Y_test['Crop'])
X_test['Season'] = label_encoder.fit_transform(X_test['Season'])
X_test['State'] = label_encoder.fit_transform(X_test['State'])


In [19]:
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(n_estimators=100,random_state=42,criterion='entropy')
classifier.fit(X_train,Y_train)

RandomForestClassifier(criterion='entropy', random_state=42)

In [20]:
Y_pred=classifier.predict(X_test)
Y_pred

array(['Pulses', 'Wheat', 'Potato', ..., 'Dry chillies', 'Ragi',
       'Moong(Green Gram)'], dtype=object)

In [21]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm=confusion_matrix(Y_test,Y_pred)
accuracy_score(Y_test,Y_pred)

0.6747079735906552

In [22]:
sample_input = {
    'Crop_Year': [2024],       # Year
    'Season': ['Kharif'],  # Season
    'State': ['Tamil Nadu'],  # Region
    'Area': [95217.0],    # Area
    'Production': [35885],
    'Annual_Rainfall':315.9 # Production
}

# Convert the sample input into a dataframe
sample_df = pd.DataFrame(sample_input)
sample_df['Season']=label_encoder.fit_transform(sample_df['Season'])
sample_df['State']=label_encoder.fit_transform(sample_df['State'])
sample_df
Y_pred_sample = classifier.predict(sample_df)
print(Y_pred_sample)

['Urad']


In [23]:
import pickle
with open('Crop_Recommend.pkl', 'wb') as f:
   pickle.dump(classifier,f)